1. IMPORT LIBRARY YANG DIPERLUKAN

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

2. NAMA FILE DAN LABELLING SESUAI HASIL SCORE KUESIONER DASS-21

In [ ]:
file_paths = {
    "D1S1.csv": "Normal",
    "D1S2.csv": "Stres Ringan",
    "D1S3.csv": "Stres Sedang",
    "D1S4.csv": "Stres Berat",
    "D2S1.csv": "Normal",
    "D2S2.csv": "Normal",
    "D2S3.csv": "Stres Ringan",
    "D2S4.csv": "Stres Sedang",
    "D3S1.csv": "Stres Ringan",
    "D3S2.csv": "Stres Sedang",
    "D3S3.csv": "Stres Ringan",
    "D3S4.csv": "Stres Ringan",
}

3. PATH DATA CSV

In [ ]:
data_dir = 'E:/SKRIPSI/Dataset/Uji PPGandGSR/'

4. MODEL YANG SUDAH DIEXPORT

In [ ]:
model_path = 'E:/SKRIPSI/Dataset/best_rf_model.pkl'


5. PROSES PREDIKSI

In [ ]:
# Mapping hasil prediksi ke label
prediction_mapping = {
    0: "Normal",
    1: "Stres Ringan",
    2: "Stres Sedang",
    3: "Stres Berat"
}

# Inisialisasi daftar untuk menyimpan hasil
all_predictions = []
true_labels = []
predicted_labels = []

# Memuat model Random Forest
model = joblib.load(model_path)

# Loop melalui setiap file untuk membaca data, agregasi, dan prediksi
for file_name, label in file_paths.items():
    # Path lengkap file
    file_path = os.path.join(data_dir, file_name)

    # Membaca data CSV
    try:
        data = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"File {file_path} tidak ditemukan, lewati...")
        continue

    # Menambahkan label
    data['Label'] = label

    # Pengecualian kolom yang tidak diperlukan
    numeric_data = data.select_dtypes(include=[np.number])

    # Menghitung mean untuk setiap fitur
    if not numeric_data.empty:
        mean_features = numeric_data.mean().to_frame().T  # Ambil rata-rata dari setiap kolom

        # Pastikan nama fitur sesuai dengan model
        mean_features.columns = [f"{col}" for col in numeric_data.columns]

        # Melakukan prediksi
        try:
            predicted_class = model.predict(mean_features)
        except ValueError as e:
            print(f"Error pada file {file_name}: {e}")
            continue

        # Mapping prediksi ke label
        predicted_label = prediction_mapping[predicted_class[0]]

        # Menambahkan hasil prediksi ke data
        mean_features['Label'] = label
        mean_features['Predicted Label'] = predicted_label

        # Menyimpan hasil ke daftar
        all_predictions.append(mean_features)
        true_labels.append(label)
        predicted_labels.append(predicted_label)
    else:
        print(f"Tidak ada data numerik valid di file: {file_name}")

# Menggabungkan semua hasil ke dalam satu DataFrame
if all_predictions:
    final_predictions = pd.concat(all_predictions, ignore_index=True)

    # Simpan hasil prediksi ke CSV
    output_file = 'E:/SKRIPSI/Dataset/Uji PPGandGSR/predicted_labels.csv'
    final_predictions.to_csv(output_file, index=False)
    print(f"Hasil prediksi disimpan di: {output_file}")

    # Menghitung akurasi dan confusion matrix
    accuracy = accuracy_score(true_labels, predicted_labels)
    conf_matrix = confusion_matrix(true_labels, predicted_labels, labels=list(set(file_paths.values())))

    # Menampilkan hasil
    print(f"Akurasi: {accuracy:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)

    # Plot confusion matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=list(set(file_paths.values())), yticklabels=list(set(file_paths.values())))
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()
else:
    print("Tidak ada hasil yang dapat diproses.")